In [64]:
import pandas as pd
from datetime import datetime, timedelta
from os import path
import glob

In [62]:
date = datetime.today() - timedelta(days = 1)
min_date = datetime(year = 2020, month = 1, day = 22)

data_dir = "data"

while date >= min_date:
    date_formatted = date.strftime("%m-%d-%Y")
    filepath = f"{data_dir}/{date_formatted}.csv"
    if not path.isfile(filepath):    
        url = f"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_formatted}.csv"
        dataframe = pd.read_csv(url)
        dataframe.to_csv(filepath, index = False)
        print(f"Saved {filepath}")
    else:
        print(f"Skipped {filepath}")
    date = date - timedelta(days = 1)

Skipped data/03-21-2020.csv
Skipped data/03-20-2020.csv
Skipped data/03-19-2020.csv
Skipped data/03-18-2020.csv
Skipped data/03-17-2020.csv
Skipped data/03-16-2020.csv
Skipped data/03-15-2020.csv
Skipped data/03-14-2020.csv
Skipped data/03-13-2020.csv
Skipped data/03-12-2020.csv
Skipped data/03-11-2020.csv
Skipped data/03-10-2020.csv
Skipped data/03-09-2020.csv
Skipped data/03-08-2020.csv
Skipped data/03-07-2020.csv
Skipped data/03-06-2020.csv
Skipped data/03-05-2020.csv
Skipped data/03-04-2020.csv
Skipped data/03-03-2020.csv
Skipped data/03-02-2020.csv
Skipped data/03-01-2020.csv
Skipped data/02-29-2020.csv
Skipped data/02-28-2020.csv
Skipped data/02-27-2020.csv
Skipped data/02-26-2020.csv
Skipped data/02-25-2020.csv
Skipped data/02-24-2020.csv
Skipped data/02-23-2020.csv
Skipped data/02-22-2020.csv
Skipped data/02-21-2020.csv
Skipped data/02-20-2020.csv
Skipped data/02-19-2020.csv
Skipped data/02-18-2020.csv
Skipped data/02-17-2020.csv
Skipped data/02-16-2020.csv
Skipped data/02-15-2

In [70]:
data_files = glob.glob(f"{data_dir}/*.csv")

daily_dataframes = []

print(data_files)

for file in data_files:
    daily_dataframe = pd.read_csv(file)
    daily_dataframes.append(daily_dataframe)

combined_data = pd.concat(daily_dataframes, axis=0, ignore_index=True)

print(combined_data)

['data/02-25-2020.csv', 'data/03-03-2020.csv', 'data/01-27-2020.csv', 'data/03-08-2020.csv', 'data/02-19-2020.csv', 'data/02-03-2020.csv', 'data/02-06-2020.csv', 'data/02-05-2020.csv', 'data/03-07-2020.csv', 'data/03-19-2020.csv', 'data/02-13-2020.csv', 'data/02-20-2020.csv', 'data/02-15-2020.csv', 'data/02-18-2020.csv', 'data/02-01-2020.csv', 'data/02-27-2020.csv', 'data/02-08-2020.csv', 'data/03-02-2020.csv', 'data/02-10-2020.csv', 'data/01-24-2020.csv', 'data/03-04-2020.csv', 'data/01-29-2020.csv', 'data/01-22-2020.csv', 'data/02-02-2020.csv', 'data/01-28-2020.csv', 'data/03-10-2020.csv', 'data/02-07-2020.csv', 'data/02-16-2020.csv', 'data/02-14-2020.csv', 'data/03-17-2020.csv', 'data/02-28-2020.csv', 'data/03-20-2020.csv', 'data/03-11-2020.csv', 'data/03-01-2020.csv', 'data/02-22-2020.csv', 'data/03-18-2020.csv', 'data/02-12-2020.csv', 'data/03-14-2020.csv', 'data/03-13-2020.csv', 'data/03-05-2020.csv', 'data/01-25-2020.csv', 'data/01-31-2020.csv', 'data/03-15-2020.csv', 'data/03-2